In [1]:
from aimet_torch.cross_layer_equalization import equalize_model
from torchvision import models
import torch
import os
import sys
import torch.optim as optim

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(ROOT_DIR)

from utils.input_getter import mug_image_getter
import pytorch_lightning as pl
import torch.nn as nn

2025-02-28 18:31:26,819 - root - INFO - AIMET


/home/centar15-jet/LPCV_2025_T1/.venv3.10/lib/python3.10/site-packages/onnxscript/converter.py:823: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/usr/lib/python3.10/abc.py:106: FutureWarning: `NLLLoss2d` has been deprecated. Please use `NLLLoss` instead as a drop-in replacement and see https://pytorch.org/docs/main/nn.html#torch.nn.NLLLoss for more details.
  cls = super().__new__(mcls, name, bases, namespace, **kwargs)
/home/centar15-jet/LPCV_2025_T1/.venv3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class LightningModel(pl.LightningModule):
    def __init__(self, num_classes=64, learning_rate=2e-5):
        super(LightningModel, self).__init__()
        self.model = models.mobilenet_v3_small(pretrained=True)
        self.model.classifier[3] = nn.Linear(self.model.classifier[3].in_features, num_classes)
        self.criterion = nn.CrossEntropyLoss()
        self.learning_rate = learning_rate

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, labels = batch
        images, labels = images.to(self.device), labels.to(self.device)
        outputs = self(images)
        loss = self.criterion(outputs, labels)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.learning_rate)


In [3]:
image_getter = mug_image_getter()
mug_image = image_getter.get_input_torch()
print(mug_image.shape)

modelOriginal   = LightningModel.load_from_checkpoint("models/mobilenet_v3-epoch=192-train_loss=0.01.ckpt")
modelOriginal   = modelOriginal.model
modelQuantized  = LightningModel.load_from_checkpoint("models/mobilenet_v3-epoch=192-train_loss=0.01.ckpt")
modelQuantized  = modelQuantized.model


modelOriginal.eval()
modelQuantized.eval()

print(modelQuantized)
print(modelOriginal)

torch.Size([1, 3, 224, 224])


/home/centar15-jet/LPCV_2025_T1/.venv3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/centar15-jet/LPCV_2025_T1/.venv3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

In [4]:
from utils import helper
from dataset import utils as dsutils

helper.print_probablities_from_output(modelOriginal(mug_image.cuda()), dsutils.GLOBAL_CLASSES)
helper.print_probablities_from_output(modelQuantized(mug_image.cuda()), dsutils.GLOBAL_CLASSES)


Top-5 predictions for  on :
31 Cup                  100.0%
29 Bottle                 0.0%
34 Bowl                   0.0%
46 Dining Table           0.0%
51 Remote                 0.0%
Top-5 predictions for  on :
31 Cup                  100.0%
29 Bottle                 0.0%
34 Bowl                   0.0%
46 Dining Table           0.0%
51 Remote                 0.0%


In [5]:
#proba da  li radi 
param_num = 0

input_shape = (1, 3, 224, 224)
dummy_input = torch.randn(input_shape)

# print("CLE Started")

# equalize_model(modelQuantized, dummy_input = dummy_input)

# print("Cross-Layer Equalization applied successfully!")

print(modelQuantized)

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

In [6]:
import torch.nn as nn
has_bn=any(isinstance(layer,nn.BatchNorm2d) for layer in modelQuantized.modules())
print(has_bn)

True


In [7]:
helper.print_probablities_from_output(modelOriginal(mug_image.cuda()), dsutils.GLOBAL_CLASSES)
helper.print_probablities_from_output(modelQuantized(mug_image.cuda()), dsutils.GLOBAL_CLASSES)

Top-5 predictions for  on :
31 Cup                  100.0%
29 Bottle                 0.0%
34 Bowl                   0.0%
46 Dining Table           0.0%
51 Remote                 0.0%
Top-5 predictions for  on :
31 Cup                  100.0%
29 Bottle                 0.0%
34 Bowl                   0.0%
46 Dining Table           0.0%
51 Remote                 0.0%


In [8]:
# Fix posle nemam pojma kako radi

# from aimet_torch.bias_correction import correct_bias
# import torch

# # Define your model (example: ResNet18)
# model = models.resnet18(pretrained=True)

# # Define a dummy dataset (batch size = 1, shape = [1, 3, 224, 224])
# dummy_input = torch.rand(1, 3, 224, 224)

# # Apply Bias Correction
# correct_bias(model)

# print("Bias Correction applied successfully!")

In [9]:
from aimet_torch.quantsim import QuantizationSimModel
from aimet_common.defs import QuantScheme
from torchvision import transforms
from torch.utils.data import DataLoader
from dataset import DatasetReader

# Dummy input to define the model input size
dummy_input = torch.rand(1, 3, 224, 224)

transform = dsutils.transforms.RESIZE_NORMALIZE

# Step 1: Create QuantizationSimModel
sim = QuantizationSimModel(modelQuantized.cpu(), dummy_input=dummy_input,
                                     quant_scheme=QuantScheme.post_training_tf_enhanced,
                                     default_param_bw=8, default_output_bw=8)


coco_config = dsutils.COCOConfig(min_size=0)

datasetCOCO = dsutils.get_coco_dataset(config=coco_config, transform=transform, mode='val')
datasetIMGNET = dsutils.get_imgnet_dataset(config=None, transform=transform)

dataloader = DataLoader(datasetCOCO, batch_size=1, shuffle=True)

print(len(dataloader))



# Step 2: Compute Encodings (calibration)
def calibration_function(model, eval_iterations = 30, use_cuda = False):
    for i, data in enumerate(dataloader):
        image, _, _ = data
        model(image)
        if (i > eval_iterations):
            break
        if(i % 50 == 0):
            print(i)
    print(eval_iterations)
    print("cal")


2025-02-28 18:31:30,834 - Quant - INFO - No config file provided, defaulting to config file at /home/centar15-desktop1/LPCV_2025_T1/.venv310/lib/python3.10/site-packages/aimet_common/quantsim_config/default_config.json
2025-02-28 18:31:30,855 - Quant - INFO - Unsupported op type Squeeze
2025-02-28 18:31:30,855 - Quant - INFO - Unsupported op type Mean
2025-02-28 18:31:30,863 - Quant - INFO - Selecting DefaultOpInstanceConfigGenerator to compute the specialized config. hw_version:default
Total annotations before filtering: 36781
Total annotations after filtering: 23043
Total COCO annotations: 36781
Filtered annotations: 23043
23043


In [10]:
print(sim.model)

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): QuantizedConv2d(
        3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False
        (param_quantizers): ModuleDict(
          (weight): QuantizeDequantize(shape=(), qmin=-128, qmax=127, symmetric=True)
        )
        (input_quantizers): ModuleList(
          (0): QuantizeDequantize(shape=(), qmin=0, qmax=255, symmetric=False)
        )
        (output_quantizers): ModuleList(
          (0): None
        )
      )
      (1): QuantizedBatchNorm2d(
        16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True
        (param_quantizers): ModuleDict(
          (weight): None
          (bias): None
        )
        (input_quantizers): ModuleList(
          (0): None
        )
        (output_quantizers): ModuleList(
          (0): QuantizeDequantize(shape=(), qmin=0, qmax=255, symmetric=False)
        )
      )
      (2): QuantizedHardswish(
        (param_quantizers): ModuleD

In [11]:
sim.compute_encodings(calibration_function)

0
30
cal


In [12]:
print(sim.model)

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): QuantizedConv2d(
        3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False
        (param_quantizers): ModuleDict(
          (weight): QuantizeDequantize(shape=(), qmin=-128, qmax=127, symmetric=True)
        )
        (input_quantizers): ModuleList(
          (0): QuantizeDequantize(shape=(), qmin=0, qmax=255, symmetric=False)
        )
        (output_quantizers): ModuleList(
          (0): None
        )
      )
      (1): QuantizedBatchNorm2d(
        16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True
        (param_quantizers): ModuleDict(
          (weight): None
          (bias): None
        )
        (input_quantizers): ModuleList(
          (0): None
        )
        (output_quantizers): ModuleList(
          (0): QuantizeDequantize(shape=(), qmin=0, qmax=255, symmetric=False)
        )
      )
      (2): QuantizedHardswish(
        (param_quantizers): ModuleD

In [ ]:
import os
import time
import datetime
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from torchvision import models, transforms
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.profilers import PyTorchProfiler
import sys

# Import your dataset modules
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(ROOT_DIR)

import dataset.DatasetReader as DatasetReader
import dataset.utils as utils

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations
transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p = 0.3),
    transforms.ColorJitter(brightness = 0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define dataset paths
root_folder = r"../../datasets/imagenet/new_imgnet_coco"
annotation_file = os.path.join(r"../../datasets", "coco", "annotations", "instances_train2017.json")
image_dir = os.path.join(r"../../datasets", "coco", "train2017")

# Load datasets
dataset_coco = DatasetReader.COCODataset(
    annotation_file=annotation_file,
    image_dir=image_dir,
    target_classes=[s.lower() for s in utils.GLOBAL_CLASSES],
    transform=transform,
    min_size = 60
)

class_names = [s.lower().replace(' ', '_') for s in utils.GLOBAL_CLASSES]
dataset_imagenet = DatasetReader.CustomImageFolder(root_dir=root_folder, class_names=class_names, transform=transform)

dataset = torch.utils.data.ConcatDataset([dataset_coco, dataset_imagenet])

# Define DataLoader
batch_size = 128*4
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=15, persistent_workers=True, pin_memory=True,prefetch_factor=2)

#Napravi cist AIMET Model bez kvantizacije
num_classes=64


# Kreira fake model, pa cemo mu ubaciti nas


class LightningQuantModel(pl.LightningModule):
    def __init__(self, model_for_qat, num_classes=64, learning_rate=2e-5,alpha=0.7):
        super(LightningQuantModel, self).__init__()
        self.model = model_for_qat
        self.criterion = nn.CrossEntropyLoss()
        self.learning_rate = learning_rate
        self.alpha = alpha

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, labels, ds = batch 
        images, labels = images.to(self.device), labels.to(self.device)
        outputs = self(images)
        loss = self.criterion(outputs, labels)

        ds = ds.to(self.device)
        alpha_mask = (ds == 0).float() * self.alpha + (ds == 1).float() * (1 - self.alpha)
        loss = loss * alpha_mask
        loss = loss.mean()

        # if(ds == 0):
        #     loss = self.alpha*loss
        # elif(ds == 1):
        #     loss = (1 - self.alpha)*loss
        
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.learning_rate)


# Define model checkpoint callback
checkpoint_callback = ModelCheckpoint(
    dirpath="models/",
    filename="quant_mobilenet_v3-{epoch:02d}-{train_loss:.2f}",
    save_top_k=3,
    monitor="train_loss",
    mode="min"
)

from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger("logs/", name = "quant_mobilenet_v3", default_hp_metric=False)
profiler= PyTorchProfiler(on_trace_ready=torch.profiler.tensorboard_trace_handler("tb_logs/profiler0"),trace_memory=True,
                          schedule=torch.profiler.schedule(skip_first=10,warmup=1,wait=1,active=20))


quant_model = LightningQuantModel(sim.model)

trainer = Trainer(
    profiler=profiler,
    max_epochs=200,
    devices=1,  # Use only one GPU or CPU
    accelerator="auto",
    strategy="auto",  # Allows later multi-GPU setup without changing code
    precision = "16-mixed",
    callbacks=[checkpoint_callback],
    logger = logger
)

trainer.fit(quant_model, dataloader)


Total annotations before filtering: 860001
Total annotations after filtering: 204543
Total COCO annotations: 860001
Filtered annotations: 204543
2025-02-28 18:31:50,012 - pytorch_lightning.utilities.rank_zero - INFO - Using 16bit Automatic Mixed Precision (AMP)
2025-02-28 18:31:50,015 - pytorch_lightning.utilities.rank_zero - INFO - GPU available: True (cuda), used: True
2025-02-28 18:31:50,016 - pytorch_lightning.utilities.rank_zero - INFO - TPU available: False, using: 0 TPU cores
2025-02-28 18:31:50,016 - pytorch_lightning.utilities.rank_zero - INFO - HPU available: False, using: 0 HPUs
2025-02-28 18:31:50,017 - pytorch_lightning.utilities.rank_zero - INFO - You are using a CUDA device ('NVIDIA GeForce RTX 3050') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#t

2025-02-28 18:31:50.150950: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-28 18:31:50.157846: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740763910.166817   67400 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740763910.169540   67400 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-28 18:31:50.180106: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

2025-02-28 18:31:50,909 - pytorch_lightning.accelerators.cuda - INFO - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2025-02-28 18:31:50,942 - pytorch_lightning.callbacks.model_summary - INFO - 
  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | MobileNetV3      | 1.6 M  | eval 
1 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
1.6 M     Trainable params
324       Non-trainable params
1.6 M     Total params
6.335     Total estimated model params size (MB)
586       Modules in train mode
209       Modules in eval mode


/home/centar15-desktop1/LPCV_2025_T1/.venv310/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /home/centar15-desktop1/LPCV_2025_T1/src/mobilenet_v3/models exists and is not empty.


Epoch 0:   0%|          | 0/1124 [00:00<?, ?it/s] 

...

Epoch 15:  78%|███████▊  | 874/1124 [13:42<03:55,  1.06it/s, v_num=5, train_loss=0.192]  

/home/centar15-desktop1/LPCV_2025_T1/.venv310/lib/python3.10/site-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (158787870 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Epoch 17:  78%|███████▊  | 876/1124 [13:42<03:52,  1.06it/s, v_num=5, train_loss=0.180] 

/home/centar15-desktop1/LPCV_2025_T1/.venv310/lib/python3.10/site-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (158787870 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Epoch 23:   0%|          | 0/1124 [00:00<?, ?it/s, v_num=5, train_loss=0.106]           

In [ ]:
transform2 = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

In [ ]:
sim.model(transform2(mug_image))

DequantizedTensor([[-19.2462,  -5.8871, -15.6234,   1.3586, -18.5669, -12.6799,
                    -11.0949,  -5.4342,  -3.8492,  -3.3964, -19.2462,  -8.1513,
                     -6.7928, -20.3783, -13.1327, -22.8690, -10.1892,  -9.5099,
                    -13.8120, -12.2270, -22.8690, -19.2462,  -2.2643, -14.9441,
                    -17.6612,  -9.7363,  -4.5285,  -4.0757, -18.1141, -19.2462,
                     -6.3399,   2.7171, -15.6234, -19.0198,   5.8871,  -6.5664,
                     -7.4721,  -4.0757, -27.3975,  -6.5664, -23.5483, -13.3591,
                     -8.1513, -10.8684, -12.0006, -14.4913,  -2.0378, -10.1892,
                    -14.4913, -13.8120, -21.0576, -18.5669, -19.9255, -14.2648,
                    -20.3783, -14.0384, -22.1897,   0.4529, -15.3970, -14.0384,
                    -10.1892,   0.2264, -15.1705, -17.4348]],
                  grad_fn=<AliasBackward0>)

In [ ]:

# # Step 3: Export Quantized Model
os.makedirs('quantized', exist_ok=True)
sim.export(path='quantized', filename_prefix='mobilenetv3quantized', dummy_input=dummy_input)


2025-02-28 13:07:42,912 - Utils - INFO - successfully created onnx model with 106/122 node names updated
2025-02-28 13:07:42,942 - Quant - INFO - Layers excluded from quantization: []
2025-02-28 13:07:42,945 - Quant - WARNING - Quantsim export will stop exporting encodings for saving and loading in a future AIMET release.
To export encodings for saving and loading, use QuantizationSimModel's save_encodings_to_json() utility instead.


In [ ]:
helper.print_probablities_from_output(sim.model(transform2(mug_image)), dsutils.GLOBAL_CLASSES)

Top-5 predictions for  on :
34 Bowl                  94.2%
31 Cup                    4.0%
3 Airplane               1.0%
57 Sink                   0.4%
61 Vase                   0.3%


In [ ]:
import qai_hub as hub

compile_job = hub.submit_compile_job(
    model="mobilenetv3.aimet",
    device=hub.Device("Samsung Galaxy S24 (Family)"),
)
assert isinstance(compile_job, hub.CompileJob)

# Compile to a QNN Model Library
compile_job_qnn = hub.submit_compile_job(
    model="mobilenetv3.aimet",
    device=hub.Device("Samsung Galaxy S24 (Family)"),
    options="--target_runtime qnn_lib_aarch64_android --quantize_full_type int8",
)
assert isinstance(compile_job_qnn, hub.CompileJob)

Uploading mobilenetv3.aimet.zip
2025-02-26 16:36:31,620 - root - INFO - Uploading asset to https://tetrahub-qprod-userdata.s3-accelerate.amazonaws.com/models/mn74rp24n_2WWOphhfIrSwEWoZ.aimet.zip?uploadId=gCO3B5wSYPnDr1MTH4mXCmumwQFRI.7A2VrwE_0ohRROXDjdZWg4sB1hPGkxAqewGE0Zi6oLjJ8m9cIRPY.L6QkbO7OCnvMAqUCrFlqhPaZcSq1H2c8fOQIMXS68N5HoWiTrkOFrtgrDlWNj7F3KSQVE.SADCucqjYNPVV_BJkE-&partNumber=1&A


100%|██████████| 2.31M/2.31M [00:00<00:00, 5.59MB/s]

2025-02-26 16:36:32,055 - root - INFO - Successfully uploaded asset with response status: 200


Scheduled compile job (j5q09w7np) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/j5q09w7np/

Uploading mobilenetv3.aimet.zip
2025-02-26 16:36:34,592 - root - INFO - Uploading asset to https://tetrahub-qprod-userdata.s3-accelerate.amazonaws.com/models/mq2x57wln_0yRBjrhy9TLTo8S6.aimet.zip?uploadId=Bm16ZfqP28K0ppOUPLO5FdZcmPoT97mozw63SjHmLt2M3TXpmMwoJXUuwhnY83c1dl5JFwnUSy2ejGyUQ4pnsw6CBqPg.rcgmMvo18FiJXLdhkE3dj6lWHbCdU_sR4h_ez1bVOzdAW4lUmMD0niljH8kidAoW.BgrLu.VvbcHzw-&partNumber=1&A


100%|██████████| 2.31M/2.31M [00:01<00:00, 1.89MB/s]

2025-02-26 16:36:35,871 - root - INFO - Successfully uploaded asset with response status: 200


Scheduled compile job (jgl4e70j5) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jgl4e70j5/



In [ ]:
profile_job     = hub.submit_profile_job(compile_job.get_target_model(), device=hub.Device("Snapdragon 8 Elite QRD"),)

profile_job_qnn = hub.submit_profile_job(compile_job_qnn.get_target_model(), device=hub.Device("Snapdragon 8 Elite QRD"),)

Scheduled profile job (jp12q338g) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jp12q338g/

Scheduled profile job (jgdn700r5) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jgdn700r5/



In [ ]:
model_traced_original = torch.jit.trace(modelOriginal.cpu(), dummy_input)



compile_job_original = hub.submit_compile_job(
    model=model_traced_original,
    device=hub.Device("Samsung Galaxy S24 (Family)"),
    input_specs=dict(image=input_shape)
)
assert isinstance(compile_job_original, hub.CompileJob)

# Compile to a QNN Model Library
compile_job_original_qnn = hub.submit_compile_job(
    model=model_traced_original,
    device=hub.Device("Samsung Galaxy S24 (Family)"),
    input_specs=dict(image=input_shape),
    options="--target_runtime qnn_lib_aarch64_android",
)
assert isinstance(compile_job_original_qnn, hub.CompileJob)

Uploading tmp38txkz2e.pt
2025-02-26 16:36:46,195 - root - INFO - Uploading asset to https://tetrahub-qprod-userdata.s3-accelerate.amazonaws.com/models/mmd9zreon_jzDXEc6omiivcwsA.pt?uploadId=12VxES_BY3ogLc_sGVpfwyRnXTyYUAZQ9WaauKDPxfuCpRcKbv.bxrB7JgmFb64Boms.90LC.z3rhlELZYYnn48UMA.CF5eOSjXZ0WBpajWzId0slmSBePbImkKpQRaaYGe9IvzyVPdZl3.lA5D2L24hqxu9nvhSoqcyftrBs38-&partNumber=1&AWSAcces


100%|██████████| 6.51M/6.51M [00:00<00:00, 7.51MB/s]

2025-02-26 16:36:47,105 - root - INFO - Successfully uploaded asset with response status: 200


Scheduled compile job (jpvqz4zkg) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jpvqz4zkg/

Uploading tmp8kz4p03f.pt
2025-02-26 16:36:49,477 - root - INFO - Uploading asset to https://tetrahub-qprod-userdata.s3-accelerate.amazonaws.com/models/mnlx2kekn_Psy4FUQThP9wyFDh.pt?uploadId=iP.eGzVhGb04LXKfEmwEZd1HS0dw.GojWG5h1fEP1pUkVUsGRFdmd8aAqpAOsiEv3JrgTX6E4goZz1mxoDyrbZrdv1iJW5cc.2YIWgKN3qbCiyROUcmfBJVJnKinTmPgw5Xa9DtK5c1U3z2sCAYrnAqLmWhc5Kbl2DJIzeFXW4E-&partNumber=1&AWSAcces


100%|██████████| 6.51M/6.51M [00:00<00:00, 14.7MB/s]

2025-02-26 16:36:49,943 - root - INFO - Successfully uploaded asset with response status: 200


Scheduled compile job (jg90989wg) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jg90989wg/



In [ ]:
profile_job_original     = hub.submit_profile_job(compile_job_original.get_target_model(), device=hub.Device("Snapdragon 8 Elite QRD"),)

profile_job_original_qnn = hub.submit_profile_job(compile_job_original_qnn.get_target_model(), device=hub.Device("Snapdragon 8 Elite QRD"),)

Scheduled profile job (jpx9emm3p) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jpx9emm3p/

Scheduled profile job (j5mev44dp) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/j5mev44dp/



In [ ]:
quanTFLITE = profile_job.download_results('profile_results')
quanQNN    = profile_job_qnn.download_results('profile_results')

Saved profile results to /home/centar15-desktop1/LPCV_2025_T1/src/mobilenet_v3/profile_results/job_j5q09w7np_optimized_tflite_jp12q338g_results.json
Saved profile results to /home/centar15-desktop1/LPCV_2025_T1/src/mobilenet_v3/profile_results/job_jgl4e70j5_optimized_so_jgdn700r5_results.json


In [ ]:
quanTFLITE

ProfileJobResult
----------------
status                        : JobStatus
---------
code    : SUCCESS
message : 

url                           : https://app.aihub.qualcomm.com/jobs/jp12q338g/
artifacts_dir                 : /home/centar15-desktop1/LPCV_2025_T1/src/mobilenet_v3/profile_results
Estimated Inference Time (ms) : 0.196
Load Time (ms)                : 205.472
Peak Memory (MB)              : 174.859375
Compute Units (layers)        : NPU: 124

In [ ]:
quanQNN

ProfileJobResult
----------------
status                        : JobStatus
---------
code    : SUCCESS
message : 

url                           : https://app.aihub.qualcomm.com/jobs/jgdn700r5/
artifacts_dir                 : /home/centar15-desktop1/LPCV_2025_T1/src/mobilenet_v3/profile_results
Estimated Inference Time (ms) : 0.274
Load Time (ms)                : 220.315
Peak Memory (MB)              : 136.109375
Compute Units (layers)        : NPU: 113

In [ ]:
quanTFLITE_original = profile_job_original.download_results('profile_results')
quanQNN_original    = profile_job_original_qnn.download_results('profile_results')

Saved profile results to /home/centar15-desktop1/LPCV_2025_T1/src/mobilenet_v3/profile_results/job_jpvqz4zkg_optimized_tflite_jpx9emm3p_results.json
Saved profile results to /home/centar15-desktop1/LPCV_2025_T1/src/mobilenet_v3/profile_results/job_jg90989wg_optimized_so_j5mev44dp_results.json


In [ ]:
quanTFLITE_original

ProfileJobResult
----------------
status                        : JobStatus
---------
code    : SUCCESS
message : 

url                           : https://app.aihub.qualcomm.com/jobs/jpx9emm3p/
artifacts_dir                 : /home/centar15-desktop1/LPCV_2025_T1/src/mobilenet_v3/profile_results
Estimated Inference Time (ms) : 0.332
Load Time (ms)                : 313.951
Peak Memory (MB)              : 209.109375
Compute Units (layers)        : NPU: 114

In [ ]:
quanQNN_original

ProfileJobResult
----------------
status                        : JobStatus
---------
code    : SUCCESS
message : 

url                           : https://app.aihub.qualcomm.com/jobs/j5mev44dp/
artifacts_dir                 : /home/centar15-desktop1/LPCV_2025_T1/src/mobilenet_v3/profile_results
Estimated Inference Time (ms) : 0.317
Load Time (ms)                : 273.872
Peak Memory (MB)              : 162.33203125
Compute Units (layers)        : NPU: 125

In [ ]:
datasetCOCO2 = DatasetReader.COCODataset(
    annotation_file = r"/home/centar15-desktop1/LPCV_2025_T1/datasets/coco/annotations/instances_val2017.json", 
    image_dir= r'/home/centar15-desktop1/LPCV_2025_T1/datasets/coco/val2017',
    target_classes=[s.lower() for s in dsutils.GLOBAL_CLASSES],
    transform = transform,
    min_size = 60)

Total annotations before filtering: 36781
Total annotations after filtering: 8862
Total COCO annotations: 36781
Filtered annotations: 8862


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

validationDataloader = DataLoader(datasetCOCO2, batch_size = 64, shuffle = False)
with torch.no_grad():
    print("Original  model accuracy = ", helper.check_accuracy(modelOriginal.cuda(), validationDataloader, device))
    # print("Quantized model accuracy = ", helper.check_accuracy(sim.model.cuda(), validationDataloader, device))

Original  model accuracy =  0.7958700067704807


In [ ]:
with torch.no_grad():
    #print("Original  model accuracy = ", helper.check_accuracy(modelOriginal.cuda(), validationDataloader, device))
    print("Quantized model accuracy = ", helper.check_accuracy(sim.model.cuda(), validationDataloader, device))

Quantized model accuracy =  0.02561498533062514
